In [ ]:
import tarfile

In [ ]:
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [ ]:
!touch train.py

In [ ]:
!touch inference.py

In [ ]:
!touch requirements.txt

In [28]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

In [29]:
sagemaker_session.default_bucket()

'sagemaker-us-west-1-088213581398'

In [30]:
from sagemaker import get_execution_role
role = get_execution_role()

In [31]:
from sagemaker.tensorflow import TensorFlowModel

model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz', 
                        role = role, 
                        entry_point='inference.py',
                        source_dir='src',
                        framework_version='2.1')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


---------------!

In [36]:
import json
import boto3
import numpy as np
import io

endpoint_name = predictor.endpoint

client = boto3.client('runtime.sagemaker', region_name='us-west-1')

#data = [[3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5],[3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5]]
#TestData = {"instances": [{"input_token":[1,2,4,4,4,4,4,4,4,4,1,2,4,4,4,4,4,4,4,4,1,2,4,4,4,4,4,4,4,4,1,2,4,4,4,4,4,4,4,4,1,2,4,4,4,4,4,4,4,4,1,2,4,4,4,4,4,4,4,4,1,2,4,4,4,4,4,4,4,4,1,2,4,4,4,4,4,4,4,4,1,2,4,4,4,4,4,4,4,4,1,2,4,4,4,4,4,4,4,4], "masked_token":[3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,5,5,5]}]}

textext = 'the stock is great and good'
text2 = 'the stock is bad and terrible'

inp = {'instances':[{"text": 'the stock is doing great!'},{"text": 'the stock is doing terrible'}]}

payload = json.dumps(inp)

response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  ContentType='application/json',
                                  Body=payload)

response1 = response["Body"].read()
result = json.loads(response1.decode())


prediction = result['predictions']
prediction


The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


[[0.897781968, 0.00623450102, 0.0959835574],
 [0.296967685, 0.137415558, 0.565616786]]

In [27]:
predictor.endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
